# EDA con Sweetviz

Este notebook realiza un análisis exploratorio de datos automático utilizando la librería Sweetviz.

In [1]:
# Instalación de dependencias
# !pip install sweetviz

In [2]:
import pandas as pd
import sweetviz as sv

# Definir ruta al archivo (asumiendo ejecución desde la carpeta notebooks/)
DATA_PATH = "../data/02_intermediate/processed_data.parquet"

try:
    df = pd.read_parquet(DATA_PATH)
    print(f"Datos cargados correctamente. Dimensiones: {df.shape}")
except FileNotFoundError:
    print(f"No se encontró el archivo en {DATA_PATH}. Verifica la ruta.")
except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")

c:\Users\OMAR\miniconda3\envs\xgb_env\lib\site-packages\sweetviz\graph.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Datos cargados correctamente. Dimensiones: (442067, 327)


In [3]:
# Determinar variable objetivo
target_col = None
possible_targets = ['CVDINFR4', 'CVDCRHD4']

for col in possible_targets:
    if col in df.columns:
        target_col = col
        break

if target_col:
    print(f"Variable objetivo detectada: {target_col}")
else:
    print("No se detectó ninguna variable objetivo conocida. Se realizará el análisis sin target.")

Variable objetivo detectada: CVDINFR4


In [5]:
# Generar reporte
# Agregamos pairwise_analysis='off' para que no intente cruzar las 327 columnas entre sí

if target_col:
    # Si hay una columna objetivo definida
    report = sv.analyze(df, target_feat=target_col, pairwise_analysis='off')
else:
    # Si no hay columna objetivo
    report = sv.analyze(df, pairwise_analysis='off')

# Mostrar y guardar reporte
report.show_html("SWEETVIZ_REPORT.html")
print("Reporte generado: SWEETVIZ_REPORT.html")

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
Reporte generado: SWEETVIZ_REPORT.html
